# Excel报表

In [1]:
# 读取一月的交易记录
import pandas as pd
import numpy as np

# 展示内容设置为10行
pd.options.display.max_rows = 10
df = pd.read_excel('./data/sales_data/new/January.xlsx')
df.head()

,transaction_id,store,status,transaction_date,plan,contract_type,amount
0,abfbdd6d,Chicago,ACTIVE,2019-01-01,Silver,NEW,14.25
1,136a9997,San Francisco,ACTIVE,2019-01-01,Gold,NEW,19.35
2,c6688f32,San Francisco,ACTIVE,2019-01-01,Bronze,NEW,12.20
3,6ef349c1,Chicago,ACTIVE,2019-01-01,Gold,NEW,19.35
4,22066f29,San Francisco,ACTIVE,2019-01-01,Silver,NEW,14.25


In [2]:
from pathlib import Path

# 获取当前文件的目录
this_dir = Path(".").resolve()
# 从sales_data的所有子文件夹中读取Excel文件
parts = []
for path in (this_dir / "data/sales_data").rglob("*.xls*"):
    #print(f'Reading {path.name}')
    part = pd.read_excel(path, index_col="transaction_id")
    parts.append(part)
df = pd.concat(parts)
pivot = pd.pivot_table(df,
                       index='transaction_date',
                       columns="store",
                       values="amount",
                       aggfunc=sum)
# 按月采样，并赋予一个索引名称
summary = pivot.resample('M').sum()
summary.index.name = "Month"
summary.to_excel(this_dir / "sales_report_pandas.xlsx")

### read_excel函数和ExcelFile类

In [3]:
# 通过sheet_name、skiprows和usecols这些参数，告诉pandas关于我们想要读取的列的详细信息
df = pd.read_excel('./data/xl/stores.xls', sheet_name='2019', skiprows=1, usecols='B:F')
df.head()

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING
2,Chicago,4,Katelin,2020-01-31,NaN
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False


In [4]:
# 可以通过执行info方法了解生成的DataFrame的数据类型
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   5 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 368.0+ bytes


In [5]:
# Flagship应该是bool而不是object，要修正这一问题，可以编写一个转换函数
def fix_missing(x):
    return False if x in ["", "MISSING"] else x


df = pd.read_excel('./data/xl/stores.xls',
                   sheet_name='2019',
                   skiprows=1,
                   usecols='B:F',
                   converters={'Flagship': fix_missing})
df.head()

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,False
2,Chicago,4,Katelin,2020-01-31,False
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False


In [6]:
# 要读入所有的工作表，sheet_name=None
sheets = pd.read_excel('./data/xl/stores.xls',
                       sheet_name=['2019', '2020'],
                       usecols=['Store', 'Employees'],
                       skiprows=1)
sheets['2020'].head()

,Store,Employees
0,New York,11
1,San Francisco,10
2,Chicago,5
3,Boston,4
4,Washington DC,7


In [7]:
# 如果源文件的列没有标题，则设置参数header=None，并通过names参数设置对应的列名。注意sheet_name也接受工作表切片
df = pd.read_excel('./data/xl/stores.xls',
                   header=None,
                   skiprows=2,
                   skipfooter=3,
                   sheet_name=0,  # 第一个工作表
                   usecols='B:C, F',
                   names=['Branch', 'Employee_Count', 'Is_Flagship'])
df.head()

,Branch,Employee_Count,Is_Flagship
0,New York,10,False
1,San Francisco,12,MISSING
2,Chicago,4,NaN


In [8]:
# 为了处理NaN，可以把na_values和keep_default_na结合起来。
# 直将含有MISSING的单元格解释为NaN，除此之外什么也不做：
df = pd.read_excel('./data/xl/stores.xls',
                   sheet_name='2019',
                   skiprows=1,
                   usecols='B,C,F',
                   skipfooter=2,
                   na_values='MISSING',
                   keep_default_na=False)
df.head()

,Store,Employees,Flagship
0,New York,10,False
1,San Francisco,12,NaN
2,Chicago,4,
3,Boston,5,True


In [9]:
#  使用ExcelFile类来读取Excel文件
with pd.ExcelFile('./data/xl/stores.xls') as f:
    df1 = pd.read_excel(f, sheet_name='2019', skiprows=1, usecols='B:F', nrows=2)
    df2 = pd.read_excel(f, sheet_name='2020', skiprows=1, usecols='B:F', nrows=2)

In [10]:
df1

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


In [11]:
df2

,Store,Employees,Manager,Since,Flagship
0,New York,11,Sarah,2018-07-20,False
1,San Francisco,10,Neriah,2019-11-02,True


In [12]:
# 也可以通过ExcelFile防伪所有工作表的名称
stores = pd.ExcelFile('./data/xl/stores.xls')
stores.sheet_names

['2019', '2020', '2019-2020']

In [14]:
# 还可以通过URL读取Excel文件
url = "https://raw.githubusercontent.com/fzumstein/python-for-excel/1st-edition/xl/stores.xls"
pd.read_excel(url, skiprows=1, usecols='B:F', nrows=2)

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


### to_excel方法和ExcelWriter类

In [15]:
# 将DataFrame导出为Excel
import numpy as np
from datetime import datetime

data = [[datetime(2022, 5, 1, 22, 42), 2.222, 1, True],
        [datetime(2022, 5, 14, 22, 43), np.nan, 2, False],
        [datetime(2022, 5, 14, 22, 44), np.inf, True]]
df = pd.DataFrame(data, columns=['Dates', 'Floats', 'Integers', 'Booleans'])
df.index.name = 'index'
df

,Dates,Floats,Integers,Booleans
index,,,,
0,2022-05-01 22:42:00,2.222,1,True
1,2022-05-14 22:43:00,NaN,2,False
2,2022-05-14 22:44:00,inf,True,None


In [16]:
df.to_excel('write_with_pandas.xlsx',
            sheet_name='Output',
            startrow=1,
            startcol=1,
            index=True,
            header=True,
            na_rep='<NA>',
            inf_rep='<INF>')

In [17]:
# 使用ExcelWriter将DataFrame导出为Excel
with pd.ExcelWriter('written_with_pandas2.xlsx') as writer:
    df.to_excel(writer, sheet_name='sheet1', startrow=1, startcol=1)
    df.to_excel(writer, sheet_name='sheet1', startrow=10, startcol=1)
    df.to_excel(writer, sheet_name='sheet2')

# 8 使用读写包操作Excel文件

## 8.1 使用OpenPyXL读取文件

In [18]:
import openpyxl
import excel

In [19]:
# 打开工作簿，在加载数据之后文件会自动关闭
book = openpyxl.load_workbook('./data/xl/stores.xlsx', data_only=True)
sheet = book['2019']
sheet = book.sheetnames
sheet

['2019', '2020', '2019-2020']

In [20]:
# 遍历所有工作表对象
for i in book.worksheets:
    print(i.title)

2019
2020
2019-2020


In [21]:
# 获取维度
sheet = book.worksheets[0]
sheet.max_row, sheet.max_column

(8, 6)

In [22]:
# 读取某个单元格的值
sheet['B6'].value

'Boston'

In [23]:
# 或者使用单元格索引方法
sheet.cell(row=6, column=2).value

'Boston'

In [24]:
# 使用excel模块读取一个单元格区域的值，读取生成为一个列表
data = excel.read(book['2019'], (2, 2), (8, 6))
# 打印前两行
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

In [25]:
type(data)

list

### 使用OpenPyXL写入文件

In [26]:
# OpenPyXL会在内存中构建Excel文件，调用save方法时会将其写入文件
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, colors
from openpyxl.styles.borders import Border, Side
from openpyxl.styles.alignment import Alignment
from openpyxl.styles.fills import PatternFill
from openpyxl.chart import BarChart, Reference
import datetime as dt

In [27]:
# 实例化工作簿
book = openpyxl.Workbook()
# 获取第一张工作表并赋予一个名称
sheet = book.active
sheet.title = 'Sheet1'
# 使用A1表示法和单元格索引
sheet['A1'].value = 'Hello 1'
sheet.cell(row=2, column=1, value='Hello 2')
# 格式化：填充颜色、对齐、边框和字体
font_format = Font(color='FF0000', bold=True)
thin = Side(border_style='thin', color='FF0000')
sheet['A3'].value = 'Hello 3'
sheet['A3'].font = font_format
sheet['A3'].border = Border(top=thin, left=thin, right=thin, bottom=thin)
sheet['A3'].alignment = Alignment(horizontal="center")
sheet['A3'].fill = PatternFill(fgColor='FFFF00', fill_type='solid')
# 数字格式化（使用Excel的格式化字符串）
sheet['A4'].value = 3.3333
sheet['A4'].number_format = '0.00'
sheet['A5'].value = dt.date(2022, 5, 15)
sheet['A5'].number_format = 'mm/dd/yy'
# 使用公式，必须使用以逗号分隔的英文公式名称
sheet['A6'].value = 'SUM(A4, 2)'
# 加载图片
sheet.add_image(Image('./data/python.png'), 'C1')
# 二维列表（使用excel模块）
data = [[None, 'North', 'South'],
        ['Last Year', 2, 5],
        ['This Year', 3, 6]]
excel.write(sheet, data, 'A10')
# 图表
chart = BarChart()
chart.type = 'col'
chart.title = "Sales Per Region"
chart.x_axis.title = 'Region'
chart.y_axis.title = 'Sales'
# 图表数据来源
chart_data = Reference(sheet, min_row=11, min_col=1, max_row=12, max_col=3)
# 图表标题
chart_categories = Reference(sheet, min_row=10, min_col=2, max_row=10, max_col=3)
# titles_from_data 就像手动在Excel中添加图表解释数据
chart.add_data(chart_data, titles_from_data=True, from_rows=True)
chart.set_categories(chart_categories)
sheet.add_chart(chart, 'A15')
book.save(datetime.now().strftime('%Y%m%d%H%M%S_') + 'openpyxl.xlsx')

In [28]:
# 如果Excel中包含图表或者其他高级内容，那么OpenPyXL的功能就显得非常有限了，这些内容要么被修改，要么会被直接丢弃掉
# 在3.0.5版本汇总图表会被重命名且标题会被丢弃掉，
openpyxl.__version__

'3.0.9'

In [29]:
# 读取并修改stores.xlsx文件，修改一个单元格
# 并将其以最新的名称保存到新的位置
book = openpyxl.load_workbook('./data/xl/stores.xlsx')
book['2019']['A1'].value = 'modified'
book.save('stores_edited.xlsx')

In [30]:
# 如果想编写带vba格式的Excel文件，那么就必须处理一个已经存在的文件，并且在加载的时候需要将keep_vba参数设置为True
book = openpyxl.load_workbook('./data/xl/macro.xlsm', keep_vba=True)
book['Sheet1']['A1'].value = 'Click the button!'
book.save('macro_openpyxl.xlsm')

### XlsxWriter

> XlsxWriter从0开始索引单元格，OpenPyXL从1开始索引

In [31]:
import datetime as dt
import xlsxwriter
import excel

In [32]:
book = xlsxwriter.Workbook('xlsxwriter.xlsx')
# 添加工作表
sheet = book.add_worksheet('Sheet1')
# 使用A0或者单元格索引添加数据
sheet.write('A1', 'Hello 1')
sheet.write(1, 0, 'Hello 2')
# 格式化填充颜色、对齐、边框和字体
formatting = book.add_format({'font_color': 'FF0000',
                              'bg_color': 'FFFF00',
                              'bold': True,
                              'border': 1,
                              'border_color': 'FF0000'})
sheet.write('A3', 'Hello 3', formatting)
# 数字格式化
number_format = book.add_format({'num_format': '0.00'})
sheet.write('A4', 3.3333, number_format)
# 日期格式化
date_format = book.add_format({'num_format': 'mm/dd/yy'})
sheet.write('A5', dt.date(2022, 5, 15), date_format)
# 公式
sheet.write('A6', '=SUM(A4, 2)')
# 图片
sheet.insert_image(0, 2, './data/python.png')
# 二维列表
data = [[None, 'North', 'South'],
        ['LastYear', 2, 5],
        ['ThisYear', 3, 6]]
excel.write(sheet, data, 'A10')
# 图表
chart = book.add_chart({'type': 'column'})
chart.set_title({'name': 'Sales per Region'})
chart.add_series({'name': '=Sheet1!A11',
                  'categories': '=Sheet1!B10:C10',
                  'values': '=Sheet1!B 11:C11'})
chart.add_series({'name': '=Sheet1!A12',
                  'categories': '=Sheet1!B10:C10',
                  'values': '=Sheet1!B12:C12'})
chart.set_x_axis({'name': 'Region'})
chart.set_y_axis({'name': 'Sales'})
sheet.insert_chart('A15', chart)
# 关闭工作簿并在磁盘上创建文件
book.close()

/Users/summer/.conda/envs/PythonEnvironment/lib/python3.9/site-packages/xlsxwriter/chart.py:836: UserWarning: Sheetname in '=Sheet1!B 11:C11' contains spaces but isn't quoted. This may cause errors in Excel.
  warn("Sheetname in '%s' contains spaces but isn't quoted. "


### 使用xlrd读取文件

In [33]:
import xlrd
import xlwt
from xlwt.Utils import cell_to_rowcol2
import xlutils
import excel

In [34]:
# 打开工作簿来读取单元格的值
# 在加载数据后文件会自动关闭
book = xlrd.open_workbook('./data/xl/stores.xls')
book.sheet_names()

['2019', '2020', '2019-2020']

In [35]:
for sheet in book.sheets():
    print(sheet.name)

2019
2020
2019-2020


In [36]:
# 通过名称或索引（从0开始）获取工作表对象
sheet = book.sheet_by_index(0)
sheet = book.sheet_by_name('2019')
sheet.nrows, sheet.ncols

(8, 6)

In [37]:
# 使用A1表示法或者单元格索引
# 从0开始，读取各个单元格的值
# *会解包cell_to_rowcol2返回的元组以生成各个参数
sheet.cell(*cell_to_rowcol2('B3')).value

'New York'

In [38]:
# 使用excel模块读取一个区间中单元格的值
data = excel.read(sheet, 'B2')
data[:2]

[['Store', 'Employees', 'Manager', 'Since', 'Flagship'],
 ['New York', 10.0, 'Sarah', datetime.datetime(2018, 7, 20, 0, 0), False]]

### 使用xlwt写入文件

In [39]:
# 实例化工作表
book = xlwt.Workbook()
# 添加工作表
sheet = book.add_sheet('Sheet1')
# 使用A1和单元格索引
sheet.write(*cell_to_rowcol2('A1'), 'Hello 1')
sheet.write(r=1, c=0, label='Hello 2')
# 格式化：填充颜色、对齐、边框和字体
formatting = xlwt.easyxf('font: bold on, color red;'
                         'align: horiz center;'
                         'borders: top_color red, bottom_color red, right_color red, left_color red, left thin, right thin, top thin, bottom thin;'
                         'pattern: pattern solid, fore_color yellow;')
sheet.write(r=2, c=0, label='Hello 3', style=formatting)
# 数字格式化
number_format = xlwt.easyxf(num_format_str='0.00')
sheet.write(3, 0, 3.3333, number_format)
# 日期格式化
date_format = xlwt.easyxf(num_format_str='mm/dd/yy')
sheet.write(4, 0, dt.datetime(2022, 5, 15), date_format)
# 公式
sheet.write(5, 0, xlwt.Formula('SUM(A4, 2)'))
# 二维列表
data = [[None, 'North', 'South'],
        ['LastYear', 2, 5],
        ['ThisYear', 3, 6]]
excel.write(sheet, data, 'A10')
# 图片(只支持bmp)
sheet.insert_bitmap('./data/python.bmp', 0, 2)
# 将文件写入磁盘
book.save('xlwt.xls')

## 8.2 读写包的高级主题

In [40]:
# 使用OpenPyXL写入文件
# write_only=True可以让程序的内存消耗保持在较低水平，不过这个参数会通过append方法强制助航写入，并且不在允许写入单个单元格
book = openpyxl.Workbook(write_only=True)
sheet = book.create_sheet("Write_test")
# 生成一张包含1000x200的单元格的工作表
for row in range(1000):
    sheet.append(list(range(200)))
book.save('openpyxl_optimized.xls')

# 使用OpenPyXL读取大型Excel文件，应该使用read_only=True参数来加载工作簿；
# OpenPyXL不支持with语句，使用完后需要手工关闭文件
book = openpyxl.load_workbook('./data/xl/big.xlsx', read_only=True, data_only=True, keep_links=False)
book.close()

In [41]:
# 使用XlsxWriter写入文件
# 与Openpyxl类似的选项，叫做constant_memory，它也会强制逐行写入
book = xlsxwriter.Workbook("xlsxwriter_optimized.xlsx",
                           options={'constant_memory': True})
sheet = book.add_worksheet()
for row in range(1000):
    sheet.write_row(row, 0, list(range(200)))
book.close()

In [42]:
# 使用xlrd读取工作表时，可以按需加载
with xlrd.open_workbook('./data/xl/stores.xls', on_demand=True) as book:
    # 只加载第一张工作表
    sheet = book.sheet_by_index(0)

In [43]:
# 如果不想像这里的代码这样将工作簿用作上下文管理器，则需要手动调用book.release_resources()来正确关闭工作簿。
# 要搭配pandas在上下文管理器模式下使用xlrd，可以像下面这样编码
with xlrd.open_workbook('./data/xl/stores.xls', on_demand=True) as book:
    with pd.ExcelFile(book, engine='xlrd') as f:
        df = pd.read_excel(f, sheet_name=0)

In [44]:
# pandas在读取Excel文件时，默认会逐张读取工作表，在大型文件中，这个过程会花费很长时间
# 可以并行读取工作表
"""
Python中，如果想充分利用现代计算机都具备的多核处理器，就需要使用标准库中的多线程包。多线程包会生成多个并行执行任务的Python 解释器（通常一个CPU 核心一个解释器）。此时不再是逐张处理工作表，而是一个Python 解释器处理第一张工作表，与此同时另一个Python 解释器处理第二张工作表，以此类推。然而，每个额外的Python 解释器都需要一定时间启动且需要占用额外的内存，所以如果你的文件很小，那么并行读取反而可能会更慢。对于包含多个大型工作簿的大型文件的情况，多线程可以显著加快读取过程，不过这是在假定你的系统有足够的内存处理工作负载的情况下。如果像第2章所讲的那样在Binder中运行Jupyter 笔记本，那么你就没有足够的内存，因此并行化的版本会运行得更慢。在配套代码库中，你可以找到parallel_pandas.py，这是使用OpenPyXL 并行读取工作表的一种简单实现方式。
"""
import parallel_pandas

parallel_pandas.read_excel('./data/xl/stores.xlsx', sheet_name=['2019', '2020'])

{'2019':    Unnamed: 0     Unnamed: 1 Unnamed: 2 Unnamed: 3           Unnamed: 4  \
 0         NaN          Store  Employees    Manager                Since   
 1         NaN       New York         10      Sarah  2018-07-20 00:00:00   
 2         NaN  San Francisco         12     Neriah  2019-11-02 00:00:00   
 3         NaN        Chicago          4    Katelin  2020-01-31 00:00:00   
 4         NaN         Boston          5  Georgiana  2017-04-01 00:00:00   
 5         NaN  Washington DC          3       Evan                  NaN   
 6         NaN      Las Vegas         11       Paul  2020-01-06 00:00:00   
 
   Unnamed: 5  
 0   Flagship  
 1      False  
 2    MISSING  
 3        NaN  
 4       True  
 5      False  
 6      False  ,
 '2020':    Unnamed: 0     Unnamed: 1 Unnamed: 2 Unnamed: 3           Unnamed: 4  \
 0         NaN          Store  Employees    Manager                Since   
 1         NaN       New York         11      Sarah  2018-07-20 00:00:00   
 2         NaN  S

In [45]:
# 测量默认方式读取和并行读取消耗的时间
% timeit
data = pd.read_excel('./data/xl/big.xlsx', sheet_name=None, engine='openpyxl')

In [46]:
% timeit
data = None
data = parallel_pandas.read_excel('./data/xl/big.xlsx')

### 调整DataFrame在Excel中的格式

In [47]:
with pd.ExcelFile('./data/xl/stores.xlsx', engine='openpyxl') as xlFile:
    # 读取DataFrame
    df = pd.read_excel(xlFile, sheet_name='2020')
    # 获取OpenPyXL工作簿对象
    book = xlFile.book
    sheet = book['2019']
    # 读取单个值
    value = sheet['B3'].value

In [49]:
# 可以为DataFrame报表添加一个标题
with pd.ExcelWriter('panads_openpyxl.xlsx', engine='openpyxl') as writer:
    df = pd.DataFrame({'col1': [1, 2, 3, 4],
                       'col2': [5, 6, 7, 8]})
    df.index.name = 'index'
    # 写入Excel
    df.to_excel(writer, sheet_name='Sheet1', startrow=4, startcol=2)
    # 获取OpenPyXL工作簿和工作表对象
    book = writer.book
    sheet = writer.sheets['Sheet1']
    sheet['A1'].value = 'This is a Title'

In [55]:
# 调整DataFrame索引和标题的格式
df = pd.DataFrame({'col1': [1, -2], 'col2': [-3, 4]},
                  index=['row1', 'row2'])
df.index.name = 'ix'

# 格式化索引和标题
from openpyxl.styles import PatternFill

with pd.ExcelWriter('formatting_openpyxl.xlsx', engine='openpyxl') as writer:
    # 从A1开始写入
    df.to_excel(writer, startrow=0, startcol=0)
    # 将整个df以默认自定义索引/标题格式从A6处写入
    start_row, start_col = 0, 5
    # 写入df数据部分
    df.to_excel(writer,
                header=False,
                index=False,
                startrow=start_row + 1,
                startcol=start_col + 1)
    # 获取工作表对象，并创建格式
    sheet = writer.sheets['Sheet1']
    style = PatternFill(fgColor='D9D9D9', fill_type='solid')
    # 写入带样式的标题
    for i, col in enumerate(df.columns):
        sheet.cell(row=start_row + 1, column=i + start_col + 2, value=col).fill = style
    # 写入带样式的索引
    index = [df.index.name if df.index.name else None] + list(df.index)
    for i, row in enumerate(df.index):
        sheet.cell(row=i + start_row + 2, column=start_col + 1, value=row).fill = style

In [57]:
# 格式化DataFrame数据部分
from openpyxl.styles import Alignment

with pd.ExcelWriter('data_format_openpyxl.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer)
    book = writer.book
    sheet = writer.sheets['Sheet1']
    nrows, ncols = df.shape
    for row in range(nrows):
        for col in range(ncols):
            cell = sheet.cell(row=row + 2, column=col + 2)
            cell.number_format = '0.000'
            cell.alignment = Alignment(horizontal='center')

In [60]:
# 对于XlsxWriter只能对整行整列进行格式化
with pd.ExcelWriter('data_format_xlsxwriter.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer)
    book = writer.book
    sheet = writer.sheets['Sheet1']
    # 格式化
    number_format = book.add_format({'num_format': '0.000', 'align': 'center'})
    sheet.set_column(first_col=1, last_col=2, cell_format=number_format)